In [ ]:
import mysql.connector as connector

# Connect to the MySQL database server (without specifying a database initially)
connection = connector.connect(
    user="meta_mario",
    password="abcd1234",
    host="localhost"
)

cursor = connection.cursor()

# SQL commands to create the database and tables
sql_commands = """
DROP DATABASE IF EXISTS little_lemon_db;
CREATE DATABASE little_lemon_db;
USE little_lemon_db;

CREATE TABLE orders (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Order_ID INT,
    Order_Date DATE,
    Delivery_Date DATE,
    Customer_ID INT,
    Order_Status_ID INT,
    Total_Cost DECIMAL(10, 2),
    Total_Sales DECIMAL(10, 2),
    Discount DECIMAL(5, 2),
    Delivery_Cost DECIMAL(10, 2)
);

CREATE TABLE customers (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Customer_ID INT,
    Customer_Name VARCHAR(255),
    City VARCHAR(100),
    Country VARCHAR(100),
    Postal_Code VARCHAR(20),
    Country_Code VARCHAR(10),
    Email VARCHAR(255),
    Phone VARCHAR(20)
);

CREATE TABLE employees (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Employee_ID INT,
    Employee_Name VARCHAR(255),
    Role VARCHAR(100),
    Salary DECIMAL(10, 2),
    Hire_Date DATE,
    Phone VARCHAR(20),
    Email VARCHAR(255)
);

CREATE TABLE bookings (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Booking_ID INT,
    Customer_ID INT,
    Booking_Date DATE,
    Booking_Slot TIME,
    Table_No INT,
    Status VARCHAR(50)
);

CREATE TABLE menu_items (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Menu_Item_ID INT,
    Course_Name VARCHAR(255),
    Cuisine_Name VARCHAR(255),
    Starter_Name VARCHAR(255),
    Desert_Name VARCHAR(255),
    Drink VARCHAR(255),
    Sides VARCHAR(255),
    Price DECIMAL(10, 2)
);

CREATE TABLE order_items (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Order_Item_ID INT,
    Order_ID INT,
    Menu_Item_ID INT,
    Quantity INT,
    Item_Cost DECIMAL(10, 2)
);

CREATE TABLE order_status (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Order_Status_ID INT,
    Status_Name VARCHAR(50),
    Description VARCHAR(255)
);

CREATE TABLE payments (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Payment_ID INT,
    Order_ID INT,
    Payment_Date DATE,
    Payment_Method VARCHAR(50),
    Amount_Paid DECIMAL(10, 2)
);

CREATE TABLE delivery_details (
    id INT AUTO_INCREMENT PRIMARY KEY,
    Delivery_ID INT,
    Order_ID INT,
    Delivery_Date DATE,
    Delivery_Cost DECIMAL(10, 2),
    Delivery_Status VARCHAR(50)
);

CREATE TABLE IF NOT EXISTS order_view (
        Row_Number INT,
        Order_ID VARCHAR(50),
        Order_Date DATE,
        Delivery_Date DATE,
        Customer_ID VARCHAR(50),
        Customer_Name VARCHAR(255),
        City VARCHAR(100),
        Country VARCHAR(100),
        Postal_Code VARCHAR(20),
        Country_Code VARCHAR(10),
        Cost DECIMAL(10, 2),
        Sales DECIMAL(10, 2),
        Quantity INT,
        Discount DECIMAL(5, 2),
        Delivery_Cost DECIMAL(10, 2),
        Course_Name VARCHAR(255),
        Cuisine_Name VARCHAR(255),
        Starter_Name VARCHAR(255),
        Desert_Name VARCHAR(255),
        Drink VARCHAR(255),
        Sides VARCHAR(255)
    );
"""

# Execute each SQL command
for command in sql_commands.split(';'):
    if command.strip():
        cursor.execute(command)

# Commit the changes and close the connection
connection.commit()
cursor.close()
connection.close()

print("Database and tables created successfully.")


In [ ]:
import pandas as pd
import mysql.connector as connector
from datetime import datetime

# Load the data from the Excel file
file_path = r'C:\Users\bghal\Documents\Little Lemon analysis and sales report\LittleLemon_data.xlsx'
df = pd.read_excel(file_path)

# Convert 'Order Date' and 'Delivery Date' to the appropriate date format
df['Order Date'] = pd.to_datetime(df['Order Date']).dt.date
df['Delivery Date'] = pd.to_datetime(df['Delivery Date']).dt.date

# Ensure 'Order ID' and other relevant columns are treated as strings
df['Order ID'] = df['Order ID'].astype(str)
df['Customer ID'] = df['Customer ID'].astype(str)

# Replace NaN values with None (which translates to NULL in MySQL)
df = df.where(pd.notnull(df), None)

# Connect to the MySQL database
connection = connector.connect(
    user="meta_mario",
    password="abcd1234",
    host="localhost",
    database="little_lemon_db"
)
cursor = connection.cursor()

# Insert each row from the DataFrame into the `order_view` table
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO order_view (
            `Row_Number`, `Order_ID`, `Order_Date`, `Delivery_Date`, `Customer_ID`, `Customer_Name`, 
            `City`, `Country`, `Postal_Code`, `Country_Code`, `Cost`, `Sales`, `Quantity`, `Discount`, 
            `Delivery_Cost`, `Course_Name`, `Cuisine_Name`, `Starter_Name`, `Desert_Name`, `Drink`, `Sides`
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Row Number'], row['Order ID'], row['Order Date'], row['Delivery Date'], row['Customer ID'], 
        row['Customer Name'], row['City'], row['Country'], row['Postal Code'], row['Country Code'], 
        row[' Cost'], row['Sales'], row['Quantity'], row['Discount'], row['Delivery Cost'], 
        row['Course Name'], row['Cuisine Name'], row['Starter Name'], row['Desert Name'], 
        row['Drink'], row['Sides']
    ))

# Commit the changes and close the connection
connection.commit()
cursor.close()
connection.close()

print("Data has been successfully inserted into the 'order_view' table.")
